In [1]:
import torch

In [2]:
import torchvision

In [3]:
import torch.nn as nn

In [4]:
import torchvision.transforms as transforms

In [5]:
import numpy as np

In [6]:
x = torch.tensor(1, requires_grad=True)

In [7]:
w = torch.tensor(2, requires_grad=True)

In [8]:
b = torch.tensor(3, requires_grad=True)

In [9]:
y = w*x +b

In [10]:
y.backward()

In [11]:
x.grad, w.grad, b.grad

(tensor(2), tensor(1), tensor(1))

In [12]:
x = torch.randn(10,3)

In [13]:
y = torch.randn(10,2)

In [14]:
linear = nn.Linear(3,2)

In [15]:
linear.weight, linear.bias, linear.parameters()

(Parameter containing:
 tensor([[-0.4692,  0.0975,  0.4908],
         [ 0.1454,  0.1482,  0.5002]]), Parameter containing:
 tensor([-0.2400, -0.0492]), <generator object Module.parameters at 0x1164e85c8>)

In [16]:
criterion = nn.MSELoss()

In [17]:
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

In [18]:
pred = linear(x)

In [19]:
pred, y

(tensor([[ 0.1036,  0.3680],
         [-0.0733, -0.0439],
         [ 0.7113,  0.5074],
         [-0.7180, -0.3224],
         [ 0.9196,  0.3205],
         [ 0.3931,  0.9207],
         [ 0.3907,  0.8671],
         [-1.2660, -1.2236],
         [-0.1623, -0.3927],
         [ 0.3761,  0.8501]]), tensor([[-0.1815,  0.4029],
         [ 1.3756,  1.1827],
         [-0.5060,  0.5151],
         [ 1.0946, -1.3297],
         [-0.8204,  1.2048],
         [ 1.7834, -0.1989],
         [ 0.4644, -0.4217],
         [ 0.2007, -1.3568],
         [ 1.8645,  0.0877],
         [ 2.1010, -0.1999]]))

In [20]:
loss = criterion(pred, y)

In [21]:
loss.item()

1.4357632398605347

In [22]:
loss.backward()

In [23]:
linear.weight.grad, linear.bias.grad

(tensor([[-0.3665,  0.7079,  0.1725],
         [ 0.3830, -0.1340,  0.4466]]), tensor([-0.6702,  0.1965]))

In [24]:
optimizer.step()

In [25]:
linear.weight, linear.bias

(Parameter containing:
 tensor([[-0.4655,  0.0904,  0.4890],
         [ 0.1416,  0.1495,  0.4957]]), Parameter containing:
 tensor([-0.2333, -0.0512]))

In [26]:
pred= linear(x)

In [27]:
loss = criterion(pred, y)

In [28]:
loss.item()

1.4206663370132446

In [29]:
x= np.array([[1,2],[3,4]])

In [30]:
y = torch.from_numpy(x)

In [31]:
z = y.numpy()

In [32]:
x,y,z

(array([[1, 2],
        [3, 4]]), tensor([[ 1,  2],
         [ 3,  4]]), array([[1, 2],
        [3, 4]]))

In [36]:
resnet = torchvision.models.resnet18(pretrained=True) # this causes downloading if /home/users/you/.torch/models has no corresponding model

In [37]:
#resnet = torchvision.models.resnet18(pretrained=False) # this uses a randomly initialized model

In [56]:
for params in resnet.parameters(): # freeze all params and only finetune last layer (fc)
    params.requires_grad = False
    #print(params.shape)

In [43]:
params.shape

torch.Size([1000])

In [46]:
resnet.fc = nn.Linear(resnet.fc.in_features, 100) # simply replace the top layer for finetuning

In [47]:
images = torch.randn(64, 3, 224, 224)

In [48]:
outputs = resnet(images)

In [55]:
outputs.size()

torch.Size([64, 100])

In [72]:
torch.save(resnet.state_dict(), 'resnet18_fc_finetune.ckpt')

In [73]:
resnet.load_state_dict(torch.load('resnet18_fc_finetune.ckpt'))

In [74]:
!git commit -m "getting familiar with basics of pytorch; tensor, auto grad, dataSet, dataLoader and save&load state_dict"

On branch master
Your branch is up-to-date with 'origin/master'.
Changes not staged for commit:
	modified:   ../../../.gitignore

Untracked files:
	pytorch_basics_main.ipynb

no changes added to commit


In [77]:
!git add 

fatal: pathspec 'all' did not match any files
